# Connectome Manipulation : Connection Removal

## Introduction

Welcome to this demonstration on manipulating connectomes within SONATA circuit models. This Jupyter notebook serves as a hands-on guide to understanding and implementing techniques for the modification of neural circuitry, with a particular focus on the removal of connections. Our exploration is designed to equip researchers and practitioners with the necessary tools and knowledge to advance their work in computational neuroscience and related fields.

We will be removing 20% of connections between excitatory neurons in the circuit and 30% of the Inhibitory to Excitatory connections.

### Step 1: Download example circuit

Download and uncompress SONATA network model of the rat somatosensory cortex from ZENODO (DOI [10.5281/zenodo.7930275](https://zenodo.org/doi/10.5281/zenodo.7930275)).


### Step 2: Load necessary libraries and investigate the circuit content


In [1]:
""" Global imports """
import json
import os
from pprint import pprint

""" Local imports """

from connectome_manipulator.connectome_manipulation.manipulation import Manipulation
# from connectome_manipulator.connectome_manipulation import connectome_manipulation
# from connectome_manipulator.connectome_comparison import structural_comparator
from pathlib import Path

In [2]:
import connectome_manipulator

In [3]:
# import circuit
circuit_path = '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology' # adjust the circuit config path
circuit_config_path = f'{circuit_path}/circuit_config.json'
circuit_name = 'SSCx'
node_set = 'hex_0'

working_dir = './'
output_path = './output'
os.makedirs(output_path,exist_ok=True)

In [4]:
from bluepysnap import Circuit

c = Circuit(circuit_config_path)

In [5]:
c.nodes._populations

{'POm': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e775340>,
 'S1nonbarrel_neurons': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e7753a0>,
 'VPM': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e7750a0>,
 'external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e775430>,
 'external_midrange__S1nonbarrel_neurons__chemical': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e775490>}

In [6]:
c.edges._populations

{'POm__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e7759d0>,
 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e775a90>,
 'VPM__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e775280>,
 'external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e775910>,
 'external_midrange__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e775b50>,
 'midrange__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e775bb0>}

In [7]:
pprint(c.config)

{'components': {'biophysical_neuron_models_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology',
                'mechanisms_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology',
                'morphologies_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology',
                'point_neuron_models_dir': '',
                'provenance': {'atlas_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology/atlas'},
                'synaptic_models_dir': '',
                'templates_dir': ''},
 'networks': {'edges': [{'edges_file': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology/S1nonbarrel_neurons__S1nonbarrel_neurons__chemical/edges.h5',
                         'populations': {'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': {'type': 'che

In [12]:
# Model building - Configuration

model_config = {}

""" General settings """
model_config['working_dir'] = './'
model_config['out_dir'] = '.output/'
model_config['seed'] = 7531

""" Model specifications """
model_config['circuit_config'] = os.path.relpath(circuit_config_path, circuit_path)
#model_config['model'] = {'name': 'ConnProb1stOrderCA1', 'fct': {'source': 'conn_prob', 'kwargs': {'order': 1, 'sample_size': 10000}}}
model_config['model'] = {'name': f'ConnProb1stOrder_{node_set}',
                          'fct': {'source': 'conn_prob',
                                  'kwargs': {'order': 1,
                                             'sel_src': {'node_set': node_set},
                                             'sel_dest': {'node_set': node_set},
                                             'sample_size': None}}}



# Export model config for sbatch run
config_path = f'./'  # TODO do these in the model folder
Path(config_path).mkdir(parents=True, exist_ok=True)

with open(os.path.join(config_path, f'model_config__{model_config["model"]["name"]}.json'), 'w') as f:
    json.dump(model_config, f, indent=2)
print(json.dumps(model_config, indent=2))

{
  "working_dir": "./",
  "out_dir": ".output/",
  "seed": 7531,
  "circuit_config": "circuit_config.json",
  "model": {
    "name": "ConnProb1stOrder_hex_0",
    "fct": {
      "source": "conn_prob",
      "kwargs": {
        "order": 1,
        "sel_src": {
          "node_set": "hex_0"
        },
        "sel_dest": {
          "node_set": "hex_0"
        },
        "sample_size": null
      }
    }
  }
}


### Step 3: Create the Manipulation Config

Now that we know our node and edge populations in the circuit, we can start adressing them in the config file to specify what type of manipulation we want to perform

In [8]:
# Manipulation specifications
manip_config = {}

amount_pct = 100 # Amount of target neurons to rewire afferent connections to

# prob_model_file = '/gpfs/bbp.cscs.ch/project/proj112/home/kurban/hippdiss-422/ca1_ca3_to_slice10_synaptome.npz'
# delay_model_file = os.path.join(models_path, f'DistDepDelay.json')
#flatpos_model_file = os.path.join(models_path, 'SSFlatPosMappingHexO1.json')
# props_model_file = os.path.join(models_path, 'ConnPropsPerPathway.json')

#prob_fn = os.path.split(prob_model_file)[1]
#order = int(''.join(filter(str.isdigit, prob_fn[:str.find(prob_fn, 'Order')])))

manip_config['edges_popul_name'] = 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical'
manip_config['manip'] = {'name': f'{circuit_name}_remove_conns',
                         'fcts': [{'source': 'conn_rewiring',
                                   'kwargs': {'sel_src': {'node_set': node_set,'synapse_class': 'EXC'},
                                              'sel_dest': {'node_set': node_set},
                                              'syn_class': 'EXC',
                                              'keep_indegree': False,
                                              'reuse_conns': True,
                                              'gen_method': 'duplicate_randomize', 
                                              'amount_pct': amount_pct,
                                              ''
                                             }},
                                  {'source': 'conn_rewiring',
                                   'kwargs': {'sel_src': {'node_set': node_set,'synapse_class': 'INH'},
                                              'sel_dest': {'node_set': node_set},
                                              'syn_class': 'INH',
                                              'keep_indegree': False,
                                              'reuse_conns': True,
                                              'gen_method': 'duplicate_randomize', 
                                              'amount_pct': amount_pct,
                                             }},
                                 ]}

## General settings
manip_config['circuit_path'] = circuit_path
manip_config['circuit_config'] = os.path.relpath(circuit_config_path, circuit_path) # SONATA (.json) format; path rel. to 'circuit_path'
manip_config['output_path'] = f"{output_path}/{manip_config['manip']['name']}"
manip_config['seed'] = 3220
manip_config['N_split_nodes'] = 40
manip_config['working_dir'] = f'{working_dir}'


In [9]:
pprint(manip_config)

{'N_split_nodes': 40,
 'circuit_config': 'circuit_config.json',
 'circuit_path': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology',
 'edges_popul_name': 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',
 'manip': {'fcts': [{'kwargs': {'amount_pct': 100,
                                'gen_method': 'duplicate_randomize',
                                'keep_indegree': False,
                                'reuse_conns': True,
                                'sel_dest': {'node_set': 'hex_0'},
                                'sel_src': {'node_set': 'hex_0',
                                            'synapse_class': 'EXC'},
                                'syn_class': 'EXC'},
                     'source': 'conn_rewiring'},
                    {'kwargs': {'amount_pct': 100,
                                'gen_method': 'duplicate_randomize',
                                'keep_indegree': False,
                                'reus

In [11]:
# Export model config for sbatch run

with open(os.path.join(working_dir, f'manip_config.json'), 'w') as f:
    json.dump(manip_config, f, indent=2)

### Step 3: Run the connectome manipulator

Now that we defined the manipulator config file, we can call the main method of the connectome manipulator from python. 

~~~
sbatch run_rewiring_parallel.sh "manip_config.json" "/gpfs/bbp.cscs.ch/project/proj112/home/kurban/christoph_paper/connectome_manipulator/examples/remove_connections/output" 500
~~~

In [ ]:
# Connectome manipulation - Apply manipulation
## Run from notebook with the latest connectome manipulator !!! 
#connectome_manipulation.main(manip_config, do_profiling=True, do_resume=False, keep_parquet=False)

## Run using sbatch
# COMMAND: sbatch run_rewiring.sh <manip_config>.json 1 0 0

In [ ]:
from subprocess import PIPE, run

In [ ]:
# run rewiring

idx = 0
rewiring_script = '/gpfs/bbp.cscs.ch/project/proj112/home/kurban/HIPPDISS-470/connectome_manipulator/examples/working_dir/model_building/20211110-BioM_slice10/model/run_rewiring.sh'
for i in os.listdir(config_path):
    if not i.endswith('percent.json'):
        continue

    #if idx == 0:
    #    idx+=1
    #    continue

    print(idx,end=' ')
    idx+=1

    command = ['sbatch',rewiring_script,f'{config_path}/{i}','1','0','0']
    #print(command)

    result = run(command, stdout=PIPE, stderr=PIPE, universal_newlines=True)
    print(result.returncode, result.stdout, result.stderr)
    #break